# Building AI for BlackJack Game

This notebook is for explorartion purposes. I keep this notebook as a journal.

- This idea has been started with a conversation that I have made with my nephew. He asked me if we could build an AI agent that plays the best moves for blackjack. After some arguing on the topic, we have agreed that it would be overkill to train a deep net. Hence there were already best posible moves avaible. However this table only keeps the best action to play in a total matrix of player and dealer. So we have agreed to at least to construct a q-value table via reinforcement learning. We are expecting to construct a q-value table that will consists of the best move to to make (Hit,Double,Stand) in percentages. Our intension is to keep it simple so we will not implement the full rrules of the game. We will consider Ace as 1 not 11. I've learned that this is called hard totals. The avaible best moves are as follows:

![Best Action Table](best_moves_in_hard_totals.jpeg)

- So our plan is to put the percentages of each action for each state. (Q-Values)
- We are also aware of [this](https://towardsdatascience.com/reinforcement-learning-solving-blackjack-5e31a7fb371f) page. We may check it out in the future. But for learning purposes I will check it out if I geet stuck of my own.

28.05.2020

- I start right away coding the environment, I need to remember my other projects. First I will copy my abstract methods here.
- So while checking my other [projects](https://github.com/hakanonal/geodashml) I need to decide the dimensions of the table. 
    - 1 dimension is for the playes's total card.
    - 1 dimension is for the dealers open card.
    - and one dimension is for the action. which is for simplicity Hit or Stand.
- So the shape of the table is (21,10,2)
- Ok very simple q-tabled agent has been completed.
- action space 0 is stand and 1 is hit
- Ok I am currentlly stuck at play method of environment class. I need to think thourgh how should construct reward. Since getting the hihgest number would not be promoted. But I need to give reward to get more that dealer.

31.05.2020

- I have started to read [this](https://towardsdatascience.com/reinforcement-learning-solving-blackjack-5e31a7fb371f)
- I have completed reding. My First note is that in conclusion the wins and lose counts are almost same. I need to underrstand the reward mecanism better to make comments on this. 
- Writer sugested 3 addtional improvements on his code which I bealive I can apply fitst two on them. 1-do more episodes, 2-try different learning and exploration rate.
- Writer implemented the ace or 1 succesfully. I will notte that for later use.
- I have also relized that writer structured tthe q-table as 2 dim array and first dim keeps tuple as index, which is very convinient to easily expand the features of the state paramters. So I have changes it
- In terrms of reward, this example did not updated the q-table in every action but collects all the actions in a list forr every 1 episode and then loops though backwards to update. Which I think I am going to use the same tactic here. 
- I will continue later to adapt to play actions till end of the game and calculate and update the reward afterwards for each episode.

01.06.2020

- I have continues on developing agent and environment
- I have also tested my first run and startd to correct the syntax and other types of error.
- I have seen that if I convert the q-table numpy array to dictionary I understant that I need to change the state into tuple.
- However tuple values can not be updated and state must be updated all the time. 
- So I have converted the state also to dictionary.
- It seems that I am on correct path. Now since I have converted q-table to dictionary I need to init all possible values. otherwise I get the key error.

In [19]:
import numpy as np
import random
import pickle


class agent:

    def __init__(self,discount,exploration_rate,decay_factor, learning_rate):
        self.discount = discount # How much we appreciate future reward over current
        self.exploration_rate = exploration_rate # Initial exploration rate
        self.decay_factor = decay_factor
        self.learning_rate = learning_rate
        self.q_table = {}
        
    def get_next_action(self, state):
        if random.random() < self.exploration_rate: # Explore (gamble) or exploit (greedy)
            return self.random_action()
        else:
            return self.greedy_action(state)

    def greedy_action(self, state):
        return np.argmax(self.q_table[(state['player_sum'],state['dealer_sum'])])
    def random_action(self):
        return random.random() > 0.5


    def train(self, old_state, new_state, action, reward):
        
        old_state_prediction = self.q_table[(old_state['player_sum'],old_state['dealer_sum'])][action]
        new_state_prediction = self.q_table[(new_state['player_sum'],new_state['dealer_sum'])]

        old_state_prediction = ((1-self.learning_rate) * old_state_prediction) + (self.learning_rate * (reward + self.discount * np.amax(new_state_prediction)))

        self.q_table[(old_state['player_sum'],old_state['dealer_sum'])][action] = old_state_prediction
        return old_state_prediction

    def update(self, old_state, new_state, action, reward):        
        reward = self.train(old_state, new_state, action, reward)
        self.exploration_rate *= self.decay_factor
        return reward

    def save(self, file="policy"):
        fw = open(file, 'wb')
        pickle.dump(self.q_table, fw)
        fw.close()

    def load(self, file="policy"):
        fr = open(file, 'rb')
        self.q_table = pickle.load(fr)
        fr.close()        


In [20]:
import random

class environment:

    def __init__(self, config=None):

        self.config = config
        self.agent = agent(
            discount=self.config['discount'],
            exploration_rate=self.config['exploration_rate'],
            decay_factor=self.config['decay_factor'],
            learning_rate=self.config['learning_rate']
            )
        self.initGame()

    def initGame(self):
        self.state = {'player_sum':0,'dealer_sum':0} 
        self.state['player_sum'] += self.hit()
        self.state['player_sum'] += self.hit()
        self.state['dealer_sum'] += self.hit()
        self.actions_played = []

    def start(self):
        for episode in range(1,self.config['episode']+1):
            self.initGame()
            #player turn
            while True:
                action_to_play = self.agent.get_next_action(self.state)
                new_state, ended = self.play(action_to_play)
                self.actions_played.append((self.state,new_state,action_to_play))
                self.state = new_state
                if ended:
                    break

            #dealer turn
            while True:
                new_state, ended = self.playDealer()
                self.state = new_state
                if ended:
                    break

            #q-tatble update backpropogation
            reward = self.findWinner()
            for old_state,new_state,action in reversed(self.actions_played):
                reward = self.agent.update(self.state,new_state,action,reward)

    def hit(self):
        return random.randint(1,10)

    def play(self,action):
        new_state = self.state.copy()
        ended = False
        if action:
            new_state['player_sum'] += self.hit()
        else:
            ended = True
        if new_state['player_sum'] > 21:
            ended = True
        return new_state, ended
        
    def playDealer(self):
        new_state = self.state.copy()
        ended = False
        new_state['dealer_sum'] += self.hit()
        if new_state['dealer_sum'] >= 17:
            ended = True
        return new_state, ended
        
    def findWinner(self):
        # player 1 | draw 0 | dealer -1
        winner = 0
        if self.state['player_sum'] > 21:
            if self.state['dealer_sum'] > 21:
                winner = 0
            else:
                winner = -1
        else:
            if self.state['dealer_sum'] > 21:
                winner = 1
            else:
                if self.state['player_sum'] < self.state['dealer_sum']:
                    winner = -1
                elif self.state['player_sum'] > self.state['dealer_sum']:
                    winner = 1
                else:
                    winner = 0
        return winner

In [21]:
config = {
    'discount': 0.95,
    'exploration_rate':0.5,
    'decay_factor':0.999,
    'learning_rate':1,
    'episode':1000
}

e = environment(config=config)
e.start()
e.agent.save()

KeyError: (4, 22)